In [39]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import os
%matplotlib inline
import string
from tensorflow.python.client import device_lib
from tensorflow.keras.models import Sequential, load_model, clone_model
import tensorflow_hub as hub
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy, SparseCategoricalCrossentropy, CategoricalCrossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from sklearn.metrics import ConfusionMatrixDisplay, RocCurveDisplay, classification_report
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 9729884051934557160
 xla_global_id: -1,
 name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 2252026676
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 212222661965239838
 physical_device_desc: "device: 0, name: NVIDIA GeForce GTX 1650 Ti, pci bus id: 0000:01:00.0, compute capability: 7.5"
 xla_global_id: 416903419]

In [40]:
df = pd.read_csv('DBPEDIA_test.csv')

In [41]:
df.head()

,text,l1,l2,l3
0,Liu Chao-shiuan (Chinese: 劉兆玄; pinyin: Liú Zhà...,Agent,Politician,PrimeMinister
1,"Michelle Maylene (born January 20, 1987) is an...",Agent,Actor,AdultActor
2,Hirfanlı Dam is a dam in Turkey. The developme...,Place,Infrastructure,Dam
3,Grote Prijs Stad Zottegem is a single-day road...,Event,Race,CyclingRace
4,"Johannes Petrus \""Hans\"" Nijman (September 23,...",Agent,Athlete,MartialArtist


In [42]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60794 entries, 0 to 60793
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    60794 non-null  object
 1   l1      60794 non-null  object
 2   l2      60794 non-null  object
 3   l3      60794 non-null  object
dtypes: object(4)
memory usage: 1.9+ MB


In [43]:
df.head()

,text,l1,l2,l3
0,Liu Chao-shiuan (Chinese: 劉兆玄; pinyin: Liú Zhà...,Agent,Politician,PrimeMinister
1,"Michelle Maylene (born January 20, 1987) is an...",Agent,Actor,AdultActor
2,Hirfanlı Dam is a dam in Turkey. The developme...,Place,Infrastructure,Dam
3,Grote Prijs Stad Zottegem is a single-day road...,Event,Race,CyclingRace
4,"Johannes Petrus \""Hans\"" Nijman (September 23,...",Agent,Athlete,MartialArtist


In [120]:
df['Tags'] = "[" + '"' + df.l1 + '"' +  ', ' + '"' + df.l2 + '"' + ']'

In [121]:
df.head()

,text,l1,l2,l3,Tags
0,liu chaoshiuan chinese pinyin li zhoxun born m...,Agent,Politician,PrimeMinister,"[""Agent"", ""Politician""]"
1,michelle maylene born january is an actress ad...,Agent,Actor,AdultActor,"[""Agent"", ""Actor""]"
2,hirfanl dam is a dam in turkey the development...,Place,Infrastructure,Dam,"[""Place"", ""Infrastructure""]"
3,grote prijs stad zottegem is a singleday road ...,Event,Race,CyclingRace,"[""Event"", ""Race""]"
4,johannes petrus hans nijman september november...,Agent,Athlete,MartialArtist,"[""Agent"", ""Athlete""]"


In [122]:
df.Tags.iloc[0]

'["Agent", "Politician"]'

In [123]:
df.Tags = df.Tags.apply(lambda x: eval(x))

In [128]:
df.Tags.iloc[0]

['Agent', 'Politician']

In [129]:
df.head()

,text,l1,l2,l3,Tags
0,liu chaoshiuan chinese pinyin li zhoxun born m...,Agent,Politician,PrimeMinister,"[Agent, Politician]"
1,michelle maylene born january is an actress ad...,Agent,Actor,AdultActor,"[Agent, Actor]"
2,hirfanl dam is a dam in turkey the development...,Place,Infrastructure,Dam,"[Place, Infrastructure]"
3,grote prijs stad zottegem is a singleday road ...,Event,Race,CyclingRace,"[Event, Race]"
4,johannes petrus hans nijman september november...,Agent,Athlete,MartialArtist,"[Agent, Athlete]"


In [44]:
df.text = df.text.apply(lambda x: ''.join([i for i in x if i not in string.punctuation]))

In [45]:
from re import sub

In [46]:
df.text = df.text.apply(lambda x: sub('([0-9])', '', x))

In [47]:
df.text = df.text.apply(lambda x: x.lower())

In [49]:
df.text.iloc[0]

'liu chaoshiuan chinese 劉兆玄 pinyin liú zhàoxuán born may   is a taiwanese educator and politician he is a former president of the national tsing hua university – and soochow university – and a former premier of the republic of china –'

In [50]:
df.text = df.text.apply(lambda x: sub(' +', ' ', x))

In [52]:
df.text[0]

'liu chaoshiuan chinese 劉兆玄 pinyin liú zhàoxuán born may is a taiwanese educator and politician he is a former president of the national tsing hua university – and soochow university – and a former premier of the republic of china –'

In [53]:
df.text = df.text.apply(lambda x: x.encode('ascii', 'ignore').decode())

In [55]:
df.text.iloc[0]

'liu chaoshiuan chinese  pinyin li zhoxun born may is a taiwanese educator and politician he is a former president of the national tsing hua university  and soochow university  and a former premier of the republic of china '

In [56]:
df.text = df.text.apply(lambda x: sub(' +', ' ', x))

In [57]:
df.text.iloc[0]

'liu chaoshiuan chinese pinyin li zhoxun born may is a taiwanese educator and politician he is a former president of the national tsing hua university and soochow university and a former premier of the republic of china '

In [73]:
df['l1'].value_counts()

Agent             31495
Place             11574
Species            5418
Work               5300
Event              4820
SportsSeason       1483
UnitOfWork          444
TopicalConcept      198
Device               62
Name: l1, dtype: int64

In [88]:
df['l2'].value_counts()

Athlete             7855
Person              4922
Animal              3705
Building            2700
Politician          2396
                    ... 
MusicalArtist         50
RaceTrack             43
ComicsCharacter       37
VolleyballPlayer      35
Database              33
Name: l2, Length: 70, dtype: int64

In [89]:
df['l3'].value_counts()

Gymnast                  485
Mollusca                 485
Manga                    485
AcademicJournal          485
OlympicEvent             485
                        ... 
Cycad                     37
AnimangaCharacter         37
BeachVolleyballPlayer     35
CanadianFootballTeam      34
BiologicalDatabase        33
Name: l3, Length: 219, dtype: int64

In [72]:
from sklearn.preprocessing import LabelBinarizer, MultiLabelBinarizer
from sklearn.model_selection import train_test_split

In [125]:
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

In [126]:
train_text = train_df.text.to_numpy()
test_text = test_df.text.to_numpy()

In [112]:
train_text[:2]

array(['the hopman cup also known as the hyundai hopman cup for sponsorship reasons corresponds to the th edition of the hopman cup tournament between nations in mens and womens tennis nine teams participated in the world group the hopman cup was a tennis championship won by south africas amanda coetzer and wayne ferreira coetzer and ferreira defeated thailand tamarine tanasugarn and paradorn srichaphan in the final at the burswood entertainment complex in perth australia from january through january ',
       'bolitoglossinae is a subfamily of lungless salamander most of them are native to tropical areas in central and south america though a few such as batrachoseps are found in temperate regions its sister group is the extinct genus palaeoplethodon'],
      dtype=object)

In [97]:
num = [len(i.split()) for i in train_text]

In [98]:
num[:10]

[78, 39, 143, 30, 56, 43, 309, 22, 41, 169]

In [99]:
np.percentile(num, q=90)

215.0

In [100]:
np.sum(num)

4739308

In [67]:
text_vector = layers.TextVectorization(max_tokens=100000,
                                       output_sequence_length=216)

In [68]:
text_vector.adapt(train_text)

In [70]:
text_vector.get_vocabulary()[:10]

['', '[UNK]', 'the', 'in', 'of', 'and', 'a', 'is', 'was', 'to']

In [79]:
lb = MultiLabelBinarizer()

In [127]:
train_df.head()

,text,l1,l2,l3,Tags
18963,the hopman cup also known as the hyundai hopma...,Event,Tournament,TennisTournament,"[Event, Tournament]"
31903,bolitoglossinae is a subfamily of lungless sal...,Species,Animal,Amphibian,"[Species, Animal]"
51594,air seychelles is the national airline of the ...,Agent,Company,Airline,"[Agent, Company]"
4385,pyhvesi is a mediumsized lake located in the s...,Place,BodyOfWater,Lake,"[Place, BodyOfWater]"
19166,david feherty born august is a former professi...,Agent,Athlete,GolfPlayer,"[Agent, Athlete]"


In [145]:
train_label = lb.fit_transform(train_df.Tags)

In [146]:
test_label = lb.transform(test_df.Tags)

In [169]:
test_df[:20]

,text,l1,l2,l3,Tags
37217,screen singapore was held in august on the occ...,Event,SocietalEvent,FilmFestival,"[Event, SocietalEvent]"
51092,gediz niversitesi genlik ve spor kulb known as...,Agent,SportsTeam,BasketballTeam,"[Agent, SportsTeam]"
8473,cumberland head light is a lighthouse on lake ...,Place,Tower,Lighthouse,"[Place, Tower]"
1334,yakounia is a genus of evolutely coiled planis...,Species,Animal,Mollusca,"[Species, Animal]"
47822,hollywood mogul is a computer game released in...,Work,Software,VideoGame,"[Work, Software]"
14938,rangunia college bengali is a private college ...,Agent,EducationalInstitution,University,"[Agent, EducationalInstitution]"
30735,the common crane grus grus also known as the e...,Species,Animal,Bird,"[Species, Animal]"
59336,the bulgarian cup final was the th final of th...,Event,SportsEvent,FootballMatch,"[Event, SportsEvent]"
30410,asmussia murchisoniana was a devonian crustace...,Species,Animal,Crustacean,"[Species, Animal]"
35074,the kambarata dam also known as kambarata or k...,Place,Infrastructure,Dam,"[Place, Infrastructure]"


In [172]:
test_df.text.iloc[1]

'gediz niversitesi genlik ve spor kulb known as orkide gediz niversitesi for sponsorship reasons is a turkish professional basketball club based in izmir which plays turkish basketball league tbl the team was founded by gediz university in their home arena is gediz niversitesi sports hall with a capacity of seats the team sponsored by orkide which is an liquid oil company in turkey'

In [137]:
y = train_label[0]

In [138]:
y, y.shape

(array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 (79,))

In [143]:
np.expand_dims(y, axis=0)

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

In [139]:
lb.classes_, len(lb.classes_)

(array(['Actor', 'Agent', 'AmusementParkAttraction', 'Animal', 'Artist',
        'Athlete', 'BodyOfWater', 'Boxer', 'BritishRoyalty', 'Broadcaster',
        'Building', 'Cartoon', 'CelestialBody', 'Cleric',
        'ClericalAdministrativeRegion', 'Coach', 'Comic',
        'ComicsCharacter', 'Company', 'Database', 'Device',
        'EducationalInstitution', 'Engine', 'Eukaryote', 'Event',
        'FictionalCharacter', 'FloweringPlant', 'FootballLeagueSeason',
        'Genre', 'GridironFootballPlayer', 'Group', 'Horse',
        'Infrastructure', 'LegalCase', 'MotorcycleRider', 'MusicalArtist',
        'MusicalWork', 'NaturalEvent', 'NaturalPlace', 'Olympics',
        'Organisation', 'OrganisationMember', 'PeriodicalLiterature',
        'Person', 'Place', 'Plant', 'Politician', 'Presenter', 'Race',
        'RaceTrack', 'RacingDriver', 'RouteOfTransportation', 'Satellite',
        'Scientist', 'Settlement', 'SocietalEvent', 'Software', 'Song',
        'Species', 'SportFacility', 'SportsEve

In [144]:
lb.inverse_transform(np.expand_dims(y, axis=0))

[('Event', 'Tournament')]

## Create a model

In [147]:
# Set random seed
tf.random.set_seed(42)

# Set inputs
inputs = layers.Input(shape=(1), dtype=tf.string)

# Set text vectorization
text_vector = layers.TextVectorization(max_tokens=100000, 
                                       output_sequence_length=215)

text_vector.adapt(train_text)
x = text_vector(inputs)

# Set embeddings
embeddings = layers.Embedding(input_dim=100000,
                              input_length=215,
                              output_dim=32)

x = embeddings(x)

# Create layers
x = layers.Conv1D(filters=64, kernel_size=2, padding='valid', activation='relu')(x)
x = layers.GlobalMaxPool1D()(x)
outputs = layers.Dense(79, activation='sigmoid')(x)

# Pass the inputs and outputs to model
model_1 = tf.keras.Model(inputs, outputs)

In [148]:
model_1.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_2 (TextV  (None, 215)              0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 215, 32)           3200000   
                                                                 
 conv1d (Conv1D)             (None, 214, 64)           4160      
                                                                 
 global_max_pooling1d (Globa  (None, 64)               0         
 lMaxPooling1D)                                                  
                                                                 
 dense (Dense)               (None, 79)                5135  

In [149]:
model_1.compile(optimizer=Adam(),
                loss=BinaryCrossentropy(),
                metrics=['accuracy'])

In [150]:
model_1.fit(train_text, train_label,
            validation_data=(test_text, test_label),
            epochs=5)

Epoch 1/5
1520/1520 [==============================] - 33s 15ms/step - loss: 0.0824 - accuracy: 0.5668 - val_loss: 0.0415 - val_accuracy: 0.6413
Epoch 2/5
1520/1520 [==============================] - 19s 13ms/step - loss: 0.0290 - accuracy: 0.6606 - val_loss: 0.0224 - val_accuracy: 0.6489
Epoch 3/5
1520/1520 [==============================] - 19s 13ms/step - loss: 0.0170 - accuracy: 0.6703 - val_loss: 0.0170 - val_accuracy: 0.6709
Epoch 4/5
1520/1520 [==============================] - 22s 14ms/step - loss: 0.0119 - accuracy: 0.6727 - val_loss: 0.0149 - val_accuracy: 0.6553
Epoch 5/5
1520/1520 [==============================] - 22s 14ms/step - loss: 0.0089 - accuracy: 0.6726 - val_loss: 0.0137 - val_accuracy: 0.6597


In [173]:
text = 'gediz niversitesi genlik ve spor kulb known as orkide gediz niversitesi for sponsorship reasons is a turkish professional basketball club based in izmir which plays turkish basketball league tbl the team was founded by gediz university in their home arena is gediz niversitesi sports hall with a capacity of seats the team sponsored by orkide which is an liquid oil company in turkey'
pred = np.round(model_1.predict(np.expand_dims(text, axis=0)))

In [174]:
lb.inverse_transform(pred)

[('Agent', 'SportsTeam')]